<a href="https://colab.research.google.com/github/sie21/ExamenBigData/blob/master/score_lowcost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

try:
    os.remove("metastore_db/db.lck")
    os.remove("metastore_db/dbex.lck")
except:
    pass


from pyspark.sql import SparkSession

def build_spark_session(app_name, memory='4g', executors=4):
    return SparkSession.builder\
                      .appName(app_name)\
                      .config('spark.executor.memory', memory)\
                      .config('spark.executor.instances', executors)\
                      .getOrCreate()

spark_session = build_spark_session(app_name='ok-google')

from pyspark.sql import functions as f



ModuleNotFoundError: No module named 'pyspark'

1 - joindre les dataframe sur la clé ID_CLIENT en concervant tous les clients du périmètre.
2 - compter le nombre de ID_CLIENT et vérifier qu'il correspond aux nombre d'ID_CLIENT dans la variable perimètre.
3 - Caster les variables continues en double et sauvergarder alors le df obtenu dans le repertoire data sur le cluster.
4 - Pour les variables catégorielles, créer une nouvelle variable qui prend la modalité de la variable courante si elle existe et "NA" sinon.
5- Verifier la cohérence des variables continue. Par exemple pour une variable comme age mettre à -1 tous les ages <0 ou>120ans

In [ ]:
perimetre = spark_session.read.csv("data_dauphine/sample_perimetre.csv", header=True)
histo_client_raw = spark_session.read.csv("data_dauphine/sample_histo_client.csv", header=True)
histo_train_raw = spark_session.read.csv("data_dauphine/sample_histo_train.csv", header=True)
histo_lowcost_raw = spark_session.read.csv("data_dauphine/sample_histo_lowcost.csv", header=True)
visites_raw = spark_session.read.csv("data_dauphine/sample_visites.csv", header=True)

ecrire une fonction qui cast les variables ["anciennete", "recence_cmd", "AGE"] en float

le type des variables ci dessous restants inchangé
client_cols_to_keep = ["ID_CLIENT", 'LBL_STATUT_CLT','LBL_GEO_AIR',
            'LBL_SEG_COMPORTEMENTAL','LBL_GEO_TRAIN','LBL_GRP_SEGMENT_NL',
            'LBL_SEGMENT_ANTICIPATION','FLG_CMD_CARTE_1225']



In [ ]:


histo_train = cast_columns_of_df(histo_train_raw, histo_train_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')
histo_lowcost = cast_columns_of_df(histo_lowcost_raw, histo_lowcost_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')

visites = cast_columns_of_df(visites_raw, visites_raw.columns,
                             ["ID_CLIENT"], cast_type='double')

histo_client = cast_columns_of_df(histo_client_raw,
                                  ["anciennete", "recence_cmd", "AGE"],
                                  client_cols_to_keep,
                                 cast_type='double')


faire une jointure sur le champs client_id entre tous les dataframe en conservant tous les individus du fichier perimetre


continuous_columns = ['anciennete',
 'recence_cmd',
 'AGE',
 'nb_od',
 'mean_nb_passagers',
 'mean_duree_voyage',
 'mean_mt_voyage',
 'mean_tarif_loisir',
 'mean_classe_1',
 'mean_pointe',
 'mean_depart_we',
 'flg_track_nl',
 'days_since_last_visit',
 'tx_conversion']

remplacer les valeurs manquantes par '-1' sur toutes les variables categorielles et par -1 sur les variables continues.

Pour la variables LBL_GEO_TRAIN, creer une variable geo_train, qui garde les meme modalités que LBL_GEO_TRAIN
si les modalites sont dans la liste ['Toulouse', 'Lille', 'Dijon',
                                  'Lyon', 'Marseille', 'Paris',
                                  'Nice', 'Limoges','Rouen','Rennes',
                                  'Montpellier', 'Bordeaux', 'Metz',
                                  'Strasbourg']
et prend la valeur 'na' pour les modalites ['Aéroports de Paris Orly',
                                'Aéroport de Bâle-Mulhouse / Bassel',
                                'Aéroport Lille Lesquin', 'Aéroport de Rennes',
                                'Aéroport de Nantes Atlantique',
                                'Aéroport de Marseille Provence  (MRS)', 
                                'Aéroport de Bordeaux Mérignac',
                                'Aéroports de Paris Roissy-Charles-de Gaulle', 
                                "Aéroport de Nice Côte d'Azur",
                                'Aéroport de Strasbourg',
                                'Aéroport de Lyon - Saint Exupéry', 
                                'Aéroport de Toulouse Blagnac']

Quels sont les modalites des features LBL_STATUT_CLT et LBL_SEG_COMPORTEMENTAL

Regrouper les modalites de LBL_STATUT_CLT en 4 groupes grands clients, petits clients, prospect et moyen client

In [ ]:
df1.select("flg_cmd_lowcost").distinct().collect()

[Row(flg_cmd_lowcost='1.0'),
 Row(flg_cmd_lowcost='-1'),
 Row(flg_cmd_lowcost=None)]

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression


##### features engineering et modélisation

In [ ]:

def preprocessed_df(df, label="flg_cmd_lowcostIndex"):
    max_values_to_define_str_cols = 10
    id_col = 'ID_CLIENT'
    
    dty = dict(df.dtypes)
    str_cols = [k for k, v in dty.items() if v == 'string']
    str_cols.remove(id_col)
    
    for c in str_cols:
        stringIndexer = StringIndexer(inputCol=c, outputCol=c+"Index")
        model_str = stringIndexer.fit(df)
        df = model_str.transform(df).drop(c)

    input_cols = df.columns
    input_cols.remove(id_col)
    input_cols.remove(label)
    
    assembler = VectorAssembler(inputCols=input_cols,
                            outputCol="features")
    df = assembler.transform(df)
    
    featureIndexer = VectorIndexer(inputCol="features", 
                   outputCol="indexedFeatures", 
                   maxCategories=max_values_to_define_str_cols).fit(df)
    return featureIndexer.transform(df), df


data, dff = preprocessed_df(df)

data1 = data.sample(False, 0.1, 42)

#### Logistic regression

In [ ]:
lr = LogisticRegression(labelCol="flg_cmd_lowcostIndex", 
                        featuresCol="indexedFeatures",elasticNetParam=0.5)
lr_model = lr.fit(data1)

In [ ]:
lr_model.coefficients

In [ ]:
reg = lr_model.evaluate(data1)
reg.areaUnderROC

In [ ]:
reg.recallByThreshold.count()

##### Random Forest

In [ ]:
classifier = RandomForestClassifier(labelCol="flg_cmd_lowcostIndex", 
                                    featuresCol="indexedFeatures",
                                    maxDepth=15, numTrees=100)

model_rf = classifier.fit(data1)

In [ ]:
df_prediction = model_rf.transform(data1)

In [ ]:
df_prediction = df_prediction.withColumnRenamed("flg_cmd_lowcostIndex", "label")

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

In [ ]:
evaluator.evaluate(df_prediction)

In [ ]:
from pyspark.ml.linalg import Vectors
scoreAndLabels = map(lambda x: (Vectors.dense([1.0 - x[0], x[0]]), x[1]),
                     [(0.1, 0.0), (0.1, 1.0), (0.4, 0.0), (0.6, 0.0), (0.6, 1.0), (0.6, 1.0), (0.8, 1.0)])
dataset = spark_session.createDataFrame(scoreAndLabels, ["raw", "label"])
dataset.head()

In [ ]:
df_pred = df_prediction.select("label", "prediction")

In [ ]:

df_pred.groupby("label").agg(f.count("*")).toPandas()

In [ ]:
n = df_pred.count()
n1 = df_pred.filter(df_pred.label!=df_pred.prediction).count()
n2 = df_pred.filter(df_pred.label==df_pred.prediction).count()
n2/float(n) , n1/float(n)

In [ ]:
df_pred.cache()

DataFrame[label: double, prediction: double]

In [ ]:
nb_positif = df_pred.filter("label=1").count()
nb_negatif = df_pred.filter("label=0").count()
nb_negatif, nb_positif

(98128, 10331)

In [ ]:
pred_positif = df_pred.filter("prediction=1").count()
pred_negatif = df_pred.filter("prediction=0").count()

In [ ]:
vrai_positif = df_pred.filter(df_pred.label==df_pred.prediction)\
                      .filter("prediction=1").count()
faux_positif = df_pred.filter(df_pred.label!=df_pred.prediction)\
                  .filter("prediction=1").count()

In [ ]:
vrai_negatif = df_pred.filter(df_pred.label==df_pred.prediction)\
                      .filter("prediction=0").count()
faux_negatif = df_pred.filter(df_pred.label!=df_pred.prediction)\
                  .filter("prediction=0").count()

In [ ]:
recall = vrai_positif/(vrai_positif + faux_positif)
precision = sensibility = vrai_positif/ (vrai_positif + faux_negatif)
specificity = vrai_negatif/(vrai_negatif + faux_positif)
score = (vrai_positif + vrai_negatif)/n
recall, precision, specificity, score

(1.0, 0.6846384667505566, 1.0, 0.9699609990872127)

In [ ]:
df = spark_session.read.csv("data_dauphine/sample_histo_client.csv", header=True)
df.head()

Row(ID_CLIENT='000843db32fbaecfbb047ca0bb04b1f9f4d9425a', anciennete='1550', recence_cmd='36', AGE=None, LBL_STATUT_CLT='Grand', LBL_GEO_AIR='Aéroports de Paris Orly', LBL_GRP_SEGMENT_NL='Spectateur', LBL_SEG_COMPORTEMENTAL='Chasseurs Bons Plans', LBL_GEO_TRAIN='Paris', LBL_SEGMENT_ANTICIPATION='Mixte', FLG_CMD_CARTE_1225='0')

#### how to create a spark data frame

In [ ]:
from pyspark.sql import Row
import datetime

l = [(datetime.date(2018,1,3), 'Ankit',25),
     (datetime.date(2018,2,3), 'Jalfaizy',22),
     (datetime.date(2018,1,5), 'saurabh',20),
     (datetime.date(2018,1,12), 'Bala',26)]
rdd = spark_session.sparkContext.parallelize(l)
people = rdd.map(lambda x: Row(date=x[0], name=x[1], age=int(x[2])))
schemaPeople = spark_session.createDataFrame(people)
schemaPeople.toPandas()

,age,date,name
0,25,2018-01-03,Ankit
1,22,2018-02-03,Jalfaizy
2,20,2018-01-05,saurabh
3,26,2018-01-12,Bala


In [ ]:
dd = schemaPeople.select("*",
                         f.lit(datetime.date.today()).alias("date_max"))
dd.select("*", f.datediff('date_max', 'date')\
                    .alias('days_since_last_visit')).toPandas()

,age,date,name,date_max,days_since_last_visit
0,25,2018-01-03,Ankit,2018-02-21,49
1,22,2018-02-03,Jalfaizy,2018-02-21,18
2,20,2018-01-05,saurabh,2018-02-21,47
3,26,2018-01-12,Bala,2018-02-21,40


In [ ]:
# Given a dice of 6 sides we play the following game:
#   Roll the dice until you observe all 6 sides.
#   The score of the game is the sum of your rolls.
# Estimate the average game score
# Example: 2 3 6 1 4 2 3 6 4 1 2 5 --> score: 2+3+6+1+...+1+2+5
# S is a R.V. --> estimate E(S)=?


# Step 0: simulate a single roll of a dice

import random
import numpy as np


s = []
for j in range(100):
    score = []
    for i in range(100):
        one_roll = random.randint(1,6)
        score.append(one_roll)
    s.append(np.sum(np.array(score)))
np.mean(np.array(np.array(s)))
